<a href="https://colab.research.google.com/github/ScionOfHeavens/FOUNDER_TASKS/blob/master/Practice2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain_community pypdf chromadb langchain_google_genai faiss-cpu langchain-huggingface

In [ ]:
# В Colab: вставь свой HF токен (нажми Ctrl+Enter чтобы выполнить)
import os
from getpass import getpass
if not os.environ.get("HF_TOKEN"):
    os.environ["HF_TOKEN"] = getpass()

··········


In [ ]:
import os

if not os.environ.get("LANGCHAIN_TRACING_V2"):
    os.environ["LANGCHAIN_TRACING_V2"] = "true"
    os.environ["LANGCHAIN_API_KEY"] = getpass()
    os.environ["LANGCHAIN_PROJECT"] = "saiga_test"


··········


In [ ]:

os.environ["GOOGLE_API_KEY"] = getpass()

··········


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from pathlib import Path
from langchain_community.document_loaders import PyPDFLoader

folder_path = Path("/content/drive/MyDrive/Practice2Books")  # Преобразуем строку в Path
pages = []

for file_path in folder_path.glob("*.pdf"):
    loader = PyPDFLoader(file_path)
    # Если используете асинхронный метод alazy_load:
    async for page in loader.alazy_load():
        pages.append(page)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300,
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(pages)

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS # Или DuckDB

model_name = "cointegrated/rubert-tiny2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

vectorstore = FAISS.from_documents(documents=splits,
                                   embedding=embeddings)

# loaded_vectorstore = FAISS.load_local("faiss_index", embeddings)

In [ ]:
retriever = vectorstore.as_retriever()

In [ ]:
vectorstore.save_local("faiss_index")

In [ ]:
import os
from langchain_huggingface.llms.huggingface_endpoint import HuggingFaceEndpoint
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [ ]:
from langchain_huggingface import HuggingFacePipeline
hf = HuggingFacePipeline.from_model_id(
    model_id="gpt2",
    task="text-generation",
    pipeline_kwargs={"max_new_tokens": 10},
)

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
template = """Ответь на вопрос, основываясь только на следующем контексте:
{context}

Вопрос: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
from langchain_core.runnables import RunnableLambda
from transformers import AutoTokenizer

# Подбираем токенайзер под твою модель
tokenizer = AutoTokenizer.from_pretrained("NousResearch/Nous-Hermes-Llama2-13b")

MAX_TOKENS = 800  # оставляем запас под вопрос и ответ

def limit_context(inputs: dict) -> dict:
    context = inputs["context"]
    question = inputs["question"]

    # если retriever возвращает список документов
    if isinstance(context, list):
        context = " ".join([doc.page_content for doc in context])

    tokens = tokenizer.encode(context)
    if len(tokens) > MAX_TOKENS:
        # обрезаем лишние токены
        context = tokenizer.decode(tokens[:MAX_TOKENS])
    return {"context": context, "question": question}

In [ ]:
truncate_context = RunnableLambda(limit_context)

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | truncate_context
    | prompt
    | hf
    | StrOutputParser()
)

In [ ]:
print(rag_chain.invoke("Как бегать быстрее?"))

Human: Ответь на вопрос, основываясь только на следующем контексте:
произвести тем же путем. 7. Сколько времени делать кардио с целью похуде-
ния?
Нет никаких рекомендаций. Чем больше, тем лучше. Но
ровно то же действие происходит, когда вы ходите по улице,
просто с чуть меньшей интенсивностью. Просто в зале, с на- сы.
 
Как часто можно тренировать
одну и ту же мышечную группу?
 
Наука говорит, что после развивающей тренировки ак- ставится на линию бега 
 
Помогать спортсменам: 
• Возвращать правильное беговое положение как можно быстрее 
• «Лететь» над препятствием, а не вставать на него или не перепрыгивать 
• Активно контролировать приземление с переходом в бег 
• Поддерживать ритм бега

Вопрос: Как бегать быстрее?

Наука си
